# IMPORTING THE LIBRARIES

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

# IMPORTING THE DATASET

In [2]:
fashion_mnist = keras.datasets.fashion_mnist

In [3]:
(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [4]:
# scaling : 
train_images=train_images/255.0

In [5]:
test_images=test_images/255.0

In [6]:
train_images[0].shape

(28, 28)

In [7]:
train_images=train_images.reshape(len(train_images),28,28,1)

In [8]:
test_images=test_images.reshape(len(test_images),28,28,1)

In [9]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [11]:
tuner_search = RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output',project_name="Mnist Fashion")

In [12]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 41s]
val_accuracy: 0.8999999761581421

Best val_accuracy So Far: 0.9086666703224182
Total elapsed time: 00h 04m 42s


In [13]:
model = tuner_search.get_best_models(num_models=1)[0]

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 96)        2496      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 48)        115248    
_________________________________________________________________
flatten (Flatten)            (None, 19200)             0         
_________________________________________________________________
dense (Dense)                (None, 48)                921648    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                490       
Total params: 1,039,882
Trainable params: 1,039,882
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 22s 13ms/step - loss: 0.1788 - accuracy: 0.9338 - val_loss: 0.2815 - val_accuracy: 0.9048
Epoch 5/10
1688/1688 [==============================] - 22s 13ms/step - loss: 0.1478 - accuracy: 0.9441 - val_loss: 0.2871 - val_accuracy: 0.9067
Epoch 6/10
1688/1688 [==============================] - 22s 13ms/step - loss: 0.1089 - accuracy: 0.9589 - val_loss: 0.2896 - val_accuracy: 0.9097
Epoch 7/10
1688/1688 [==============================] - 22s 13ms/step - loss: 0.0929 - accuracy: 0.9664 - val_loss: 0.3528 - val_accuracy: 0.9080
Epoch 8/10
1688/1688 [==============================] - 22s 13ms/step - loss: 0.0710 - accuracy: 0.9746 - val_loss: 0.3475 - val_accuracy: 0.9110
Epoch 9/10
1688/1688 [==============================] - 22s 13ms/step - loss: 0.0622 - accuracy: 0.9778 - val_loss: 0.3970 - val_accuracy: 0.9063
Epoch 10/10
1688/1688 [==============================] - 22s 13ms/step - loss: 0.0471 - accuracy: 0.9829 - val_loss: 0.4131 